# Instantiate `DietVision` class

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%reload_ext autoreload

from diet_vision import DietVision
dv_instance = DietVision()

In [14]:
dv_instance.upload_image('fish_chips.jpg')

'/Users/EKim45/Projects/academic/nyu_diet_vision_week11/static/images/annotator_image_1690859073.png'

In [19]:
dv_instance.diet_vision_dictionary[0]['area']

1761343

In [16]:
import numpy as np

with open('../app/sam_result.npy', 'rb') as f:
    sam_result = np.load(f, allow_pickle=True).copy()
    print(sam_result[3]['area'])

993247


In [ ]:
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np 
import os
import matplotlib.pyplot as plt 
from torch.autograd import Function

zsize = 128
batch_size = 250
iterations =  500
learningRate=0.0001

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 11, stride = 4, padding = 2)
        self.conv2 = nn.Conv2d(64, 192, 5, padding = 2)
        self.conv3 = nn.Conv2d(192, 384, 3, padding = 1)
        self.conv4 = nn.Conv2d(384, 256, 3, padding = 1)
        self.conv5 = nn.Conv2d(256, 256, 3, padding = 1)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, zsize)
	
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x,indices1 = F.max_pool2d(x,(3,3),(2,2),return_indices = True)
        x = F.relu(self.conv2(x))
        x,indices2 = F.max_pool2d(x,(3,3),(2,2),return_indices = True)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x,indices3 = F.max_pool2d(x,(3,3),(2,2),return_indices = True)	
        x = x.view(x.size(0), 256 * 6 * 6)
        x = F.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x,indices1,indices2,indices3

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.dfc3 = nn.Linear(zsize, 4096)
        self.bn3 = nn.BatchNorm2d(4096)
        self.dfc2 = nn.Linear(4096, 4096)
        self.bn2 = nn.BatchNorm2d(4096)
        self.dfc1 = nn.Linear(4096,256 * 6 * 6)
        self.bn1 = nn.BatchNorm2d(256*6*6)
        self.dconv5 = nn.ConvTranspose2d(256, 256, 3, padding = 1)
        self.dconv4 = nn.ConvTranspose2d(256, 384, 3, padding = 1)
        self.dconv3 = nn.ConvTranspose2d(384, 192, 3, padding = 1)
        self.dconv2 = nn.ConvTranspose2d(192, 64, 5, padding = 2)
        self.dconv1 = nn.ConvTranspose2d(64, 3, 12, stride = 4, padding = 2)

    def forward(self,x,i1,i2,i3):	
        x = self.dfc3(x)
        x = F.relu(self.bn3(x))
        x = self.dfc2(x)
        x = F.relu(self.bn2(x))
        x = self.dfc1(x)
        x = F.relu(self.bn1(x))
        x = x.view(batch_size,256,6,6)
        x = self.dconv5(F.max_unpool2d(x,i3,kernel_size =(3,3),stride =(2,2)))
        x = F.relu(x)
        x = F.relu(self.dconv4(x))
        x = F.relu(self.dconv3(x))
        x = self.dconv2(F.max_unpool2d(x,i2,kernel_size = (3,3), stride = (2,2)))
        x = F.relu(x)
        x = self.dconv1(F.max_unpool2d(x,i1,kernel_size = 3 ,stride = 2))
        x = F.sigmoid(x)
        return x

#--------- Pytorch pseudo-code for Structure Generator ---------#
class Structure_Generator(nn.Module):
    # contains two module in sequence, an encoder and a decoder
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    def forward(self, RGB_image):
        # Encoder takes in one RGB image and 
        # output an encoded deep shape-embedding
        shape_embedding = self.encoder(RGB_image)
        
        # Decoder takes the encoded values and output  
        # multiples 2D projection (XYZ + mask)
        XYZ, maskLogit = self.decoder(shape_embedding)
        print(XYZ)
 
        return XYZ, maskLogit

# --------- Pytorch pseudo-code for training loop ----------#
# Create 2D Conv Structure generator
model = Structure_Generator()
# only need to learn the 2D structure optimizer
optimizer = optim.SGD(model.parameters())
# 2D projections from predetermined viewpoints
XYZ, maskLogit = model('../static/images/chocolate_cake.jpg')
# fused point cloud
#fuseTrans is predetermined viewpoints info
XYZid, ML = fuse3D(XYZ, maskLogit, fuseTrans)
# Render new depth images at novel viewpoints
# renderTrans is novel viewpoints info
newDepth, newMaskLogit, collision = render2D(XYZid, ML, renderTrans)
# Compute loss between novel view and ground truth
loss_depth = L1Loss()(newDepth, GTDepth)
loss_mask = BCEWithLogitLoss()(newMaskLogit, GTMask)
loss_total = loss_depth + loss_mask
# Back-propagation to update Structure Generator
loss_total.backward()
optimizer.step()

In [21]:
import requests
import json

items = range(0, 42)
response = []
for item in items:
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation,SR%20Legacy&pageSize=200&pageNumber={item}&api_key=ljOu2YIUetK44MUIP1iW7PJzRGo7jdSTTJyzHA0k'
    res1 = requests.get(url)
    res2 = json.loads(res1.text) # list length 200
    
    for food in res2:
        response.append(food['description'])

In [22]:
food_dictionary = {
    'foods': response
}

len(food_dictionary['foods'])

8229

In [23]:
# Serializing json
json_object = json.dumps(food_dictionary, indent=4)
 
# Writing to sample.json
with open('../static/js/usdaFoods.js', "w") as outfile:
    outfile.write(json_object)

In [18]:
import numpy as np
mask_dictionary = np.full((10, 10), set())

mask_dictionary[0][0].add(1)
if not mask_dictionary[0][0]:
    print('not empty')

print(mask_dictionary[0][0])

{1}


In [17]:
s1 = set([1, 2, 5, 61])
s2 = set([5, 6, 7])

s1 = s1 | s2
s1

{1, 2, 5, 6, 7, 61}

In [25]:
set1 = set()
set2 = {1, 2, 3}

set1.update(set2)
print(set1)

{1, 2, 3}
